In [ ]:
import torch

import meshzoo

from torchfem import Solid
from torchfem.materials import Isotropic

# Material model

Isotropic linear elastic stiffness tensor

In [ ]:
# Material model
material = Isotropic(E=1000.0, nu=0.3)

# 3D Cube with hexaeders

In [ ]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1

box = Solid(nodes, elements, forces, displacements, constraints, material.C())

In [ ]:
u, f = box.solve()

In [ ]:
box.plot(u=u, node_property={"Disp": u})

# 3D Cube with hexaeders and inelastic (e.g. thermal) strain

In [ ]:
# Remove constraints and set thermal strain instead
displacements = torch.zeros_like(nodes)
constraints[:, :] = False
constraints[nodes[:, 0] == 0.0, 0] = True
constraints[nodes[:, 1] == 0.0, 1] = True
constraints[nodes[:, 2] == 0.0, 2] = True
strains = torch.stack(len(elements) * [torch.tensor([0.1, 0.2, 0.3, 0.0, 0.0, 0.0])])

box = Solid(nodes, elements, forces, displacements, constraints, material.C(), strains)

In [ ]:
u, f = box.solve()

In [ ]:
box.plot(u=u, node_property={"Disp": u})

# 3D Cube with tetrahedrons

In [ ]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1


box = Solid(nodes, elements, forces, displacements, constraints, material.C())

In [ ]:
u, f = box.solve()

In [ ]:
box.plot(u=u, node_property={"Disp": u})